# 1 Feature Engineering



Para fazer a limpeza decidi criar uma função chamada clean_data() para ser utilizado tanto com o dataframe de treino quanto com os dados do desafio após. 

Iniciei com a coluna "Certificate", uniformizando o formato das strings e denominando "Unknown" os valores NaN.

Para a coluna "Released_Year" fiz a mudança de "PG" para "1995", possível erro do dataset descoberto durante análise dos dados, e então a transformação de string para inteiro.

Na coluna "Gross" removi as strings de vírgula e transformei os dados em float caso não nulos.

Em "Runtime" removi o que não era número da string e mudei o tipo de dado para float.

Agora já partindo para colunas categóricas, devido a grande extensão de diretores evidenciei os 50 de maior frequência no dataframe e o restante denominei "Other_Directors".

Com as 4 colunas de "Star1" a "Star4", juntei os atores em uma única coluna "Star" em forma de lista, cada filme contendo uma lista dos atores. Então decidi permanecer no dataframe apenas os 100 mais comuns e o restante se tornou "Other_Stars". Com estes dados utilizei o MultiLabelBinarizer, transformando cada ator em uma coluna com uma matriz de dados binários, 0 e 1, para caso estejam presentes nos filmes ou não.

Em seguida fiz o mesmo processo com a coluna "Genre", transformando os dados em uma lista, pegando os 15 mais comuns e tornando o resto em "Other_Genres", passando também pelo processo do MultiLabelBinarizer. No fim juntei tanto as novas colunas MLB de atores quanto de gêneros ao dataframe e removi as colunas "Star1" a "Star4", "Stars", "Genre" e "Series_Title".

Por último, padronizei as strings da coluna "Overview" para se apresentarem em letra minúscula.

Aqui também já preparei a função fix_dataframe(), para igualar as colunas do dataframe do desafio com as colunas preparadas aqui na etapa de Featurne Engineering, como as colunas criadas com o MultiLabelBinarizer, e preenchendo com 0 caso esses dados não estejam presentes no novo dataframe.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [89]:
def clean_data(df):

  df['Certificate'] = df['Certificate'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else 'Unknown')


  df.loc[df['Released_Year'] == 'PG', 'Released_Year'] = 1995
  
  df['Released_Year'] = df['Released_Year'].apply(lambda x: int(str(x)) if pd.notna(x) else np.nan)


  df['Gross'] = df['Gross'].apply(lambda x: float(str(x).replace(',','')) if pd.notna(x) else np.nan)


  df['Runtime'] = df['Runtime'].str.extract(r'(\d+)').astype(float)


  top_directors = df['Director'].value_counts().head(50).index.tolist()

  df['Director'] = df['Director'].apply(lambda x: x if x in top_directors else 'Other_Directors')


  df['Stars'] = df[["Star1", "Star2", "Star3", "Star4"]].values.tolist()

  all_stars = pd.Series([s for stars in df['Stars'] for s in stars]).value_counts().head(100).index.tolist()
  
  df['Stars'] = df['Stars'].apply(lambda x: [s if s in all_stars else 'Other_Stars' for s in x])

  mlb_star = MultiLabelBinarizer()

  stars = pd.DataFrame(mlb_star.fit_transform(df['Stars']), columns=mlb_star.classes_,index=df.index)

  
  df['Genre'] = df['Genre'].str.split(', ')

  all_genres = pd.Series([g for genres in df['Genre'] for g in genres]).value_counts().head(15).index.tolist()
  
  df['Genre'] = df['Genre'].apply(lambda x: [g if g in all_genres else 'Other_Genres' for g in x])

  mlb_genre = MultiLabelBinarizer()

  genres = pd.DataFrame(mlb_genre.fit_transform(df['Genre']), columns=mlb_genre.classes_,index=df.index)

 
  df = pd.concat([df,genres,stars],axis=1)


  df["Overview"] = df["Overview"].str.lower()

  df.drop(columns=["Genre","Star1", "Star2", "Star3", "Star4"],axis=1,inplace=True)


  df.drop(columns=["Series_Title","Stars"],axis=1,inplace=True)

  return df


def fix_dataframe(new_df, expected_columns, fill_value=0):
  
  for col in expected_columns:
    if col not in new_df.columns:
      new_df[col] = fill_value
  
  new_df = new_df[expected_columns]

  return new_df  

# 2 Modelo

Como o modelo de machine learning é responsável de prever a nota do IMDB, uma variável contínua, estamos falando de um problema de regressão. 

Para atingir esse objetivo, separei os dados em:
- Features Numéricas: 'Released_Year','Runtime','Meta_score','No_of_Votes','Gross'
- Features Categóricas: 'Certificate','Director'
- Features de texto: 'Overview'
- Features Multi-Label Binarizer: Matrizes binárias transformadas previamente em colunas de gêneros e atores
O restante dos dados do dataframe, o objetivo da predição 'IMDB_Rating', não foi utilizado aqui.

Na fase de transformações, criei uma pipeline para cada seção de features:
- Features Numéricas: SimpleImputer() com mediana para preencher valores faltantes e o StandardScaler() para normalização das diferentes escalas numéricas
- Features Categóricas: SimpleImputer() com o valor mais comum, mesmo já assegurando que não haveriam mais dados nulos aqui, e OneHotEncoder() para converter as categorias em variáveis binárias e lidar com valores desconhecidos.
- Features de texto: TfidVectorizer() para capturar a importância das palavras de acordo com o contexto, limitando a dimensionalidade para 500 para evitar overfitting e removendo palavras irrelevantes em inglês, como preposições e artigos 
- Features Multi-Label Binarizer: O termo "passthrough" pois os dados já estão transformados
Com os steps designados, finalizei a transformação com ColumnTransformer().

Por fim, para completar a pipeline da machine learning com um modelo de regressão, escolhi o Random Forest Regressor, propício para prever valores numérics contínuos. Entre seus prós estão que o algorítmo captura relações complexas entre variáveis, lida bem com outliers, reduz overfitting através de suas múltiplas árvores e é versátil com os diferentes tipos de dados. Já seus contras se destacam a consumação de memória caso tenha muitas árvores e que não foge muito dos valores da faixa limitada no treino.
As alternativas seriam Linear Regression (o dataset é muito complexo), XGBoost (Random Forest se apresenta suficiente e mais adequado pelo tamanho do dataset, tempo de otimização, recursos e performance) e Redes Neurais (o dataset é muito pequeno).

Enfim, fiz a separação dos dados em 70% para treinamento e 30% para teste e apliquei a pipeline nos dados.

As métricas utilizadas para medir a performance foram o Mean Absolute Error (MAE), Mean Squarred Error (MSE), Root Mean Squared Error (RMSE) e Coeficiente de Determinação (R²), ideais para modelos de regressão. O MAE é uma medida mais interpretável, o MSE e o RMSE penalizam mais os erros maiores (como outliers) e o R² explica quanto da variabilidade dos dados o modelo consegue explicar.

Como resultado da performance do modelo:
- MAE: 0.16, apresentando uma boa precisão
- MSE: 0.04, indicando poucos erros grandes 
- RMSE: 0.20, mostrando que os erros se concentram em torno de 0.2 das notas do IMDB 
- R²: 0.50, significando que o modelo captura metade da variabilidade

Um exemplo de predição com os dados de treino e teste é o filme The Best Years of Our Lives, que com a nota 8.0 teve a previsão de nota 8.0 (7.989, arredondado)

In [ ]:
df = pd.read_csv('desafio_indicium_imdb.csv').drop('Unnamed: 0',axis=1)

df = clean_data(df)


numeric_features = ['Released_Year','Runtime','Meta_score','No_of_Votes','Gross']
categorical_features = ['Certificate','Director']
text_features = 'Overview'
mlb_features = [c for c in df.columns if c not in numeric_features + categorical_features + [text_features,'IMDB_Rating']]


numeric_transformer = Pipeline(steps=[
                      ("imputer",SimpleImputer(strategy="median")),
                      ("scaler",StandardScaler())
                      ])

categorial_transformer = Pipeline(steps=[
                        ("imputer",SimpleImputer(strategy="most_frequent")),
                        ("encoder",OneHotEncoder(handle_unknown="ignore"))
                        ])

text_transformer = Pipeline(steps=[
                  ("vectorizer",TfidfVectorizer(max_features=500,stop_words="english"))
])

mlb_transformer = "passthrough"


preprocessor = ColumnTransformer(
              transformers=[
                ("num",numeric_transformer,numeric_features),
                ("cat",categorial_transformer,categorical_features),
                ("txt",text_transformer,text_features),
                ("mlb",mlb_transformer,mlb_features)
              ],
              remainder="drop"
)


pipeline = Pipeline(steps=[
          ("preprocessor",preprocessor),
          ("regressor",RandomForestRegressor(n_estimators=200,random_state=42))
])


X = df.drop(columns=["IMDB_Rating"])
y = df['IMDB_Rating']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

pipeline.fit(X_train,y_train)

y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,y_pred)


print(f"Exemplo dado de teste 1: {y_test[:1]}")
print(f"Exemplo de predição 1: {y_pred[:1]}")
print(f"Erro absoluto médio: {mae:.2f}")
print(f"Erro quadrático médio: {mse:.2f}")
print(f"Raiz do erro quadrático médio: {rmse:.2f}")
print(f"Coeficiente de determinação: {r2:.2f}")

Exemplo dado de teste 1: 453    8.0
Name: IMDB_Rating, dtype: float64
Exemplo de predição 1: [7.989]
Erro absoluto médio: 0.16
Erro quadrático médio: 0.04
Raiz do erro quadrático médio: 0.20
Coeficiente de determinação: 0.50


# 3 Salvar o modelo

Utilizar Pickle para salvar a pipeline e as colunas que devem estar presentes no dataframe para serem utilizadas com novos dados.

In [107]:
import pickle 
final_model = {
  'pipeline': pipeline,
  'expected_columns': X_train.columns.tolist(),
}

with open('model_indicium.pkl','wb') as f:
  pickle.dump(final_model,f)

# 4 Utilizar modelo pré-treinado

Importei as funções de limpeza de dados e conformização do dataframe para que possa ser utilizado com o modelo

Importei também o modelo pré-treinado e as "expected_columns"

Então transformei os novos dados em um dataframe e o utilizei as funções para que então pudesse fazer a predição da nota do IMDB, tendo como resultado a nota 8.7865.

In [108]:
import data_cleaner

with open('model_indicium.pkl','rb') as f:
  model = pickle.load(f)

test_movie = pd.DataFrame([{'Series_Title': 'The Shawshank Redemption', 
                            'Released_Year': '1994', 
                            'Certificate': 'A', 
                            'Runtime': '142 min', 
                            'Genre': 'Drama', 
                            'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 
                            'Meta_score': 80.0, 
                            'Director': 'Frank Darabont', 
                            'Star1': 'Tim Robbins', 
                            'Star2': 'Morgan Freeman', 
                            'Star3': 'Bob Gunton', 
                            'Star4': 'William Sadler', 
                            'No_of_Votes': 2343110, 
                            'Gross': '28,341,469'
}])


df_test = data_cleaner.clean_data(test_movie)

movie = data_cleaner.fix_dataframe(df_test,model['expected_columns'])

IMDB_Rate = model['pipeline'].predict(movie)

print(f"Nota do IMDB: {IMDB_Rate}")


Nota do IMDB: [8.7865]
